# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [47]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

592

In [48]:
# Perform API Calls
# Perform a weather check on each city using a series of successive API calls.
#* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [49]:
url = "http://api.openweathermap.org/data/2.5/weather"
query = "Nashville"
url_params = {
    "api" : api,
    "q" : query,
    'page': 2
}
city_name = []
Lat = []
Long = []
MaxTemp = []
Humidity = []
Cloundiness = []
WindSpeed = []
Date = []
#enum
for city in cities:
   
    try: 
        # Build partial query URL
        query_url = f"{url}?appid={api}&q={city}"
        #query_url
    

    
        print("City : ",city)
        city_weather = requests.get(query_url).json()
    
    
    
        print(city_weather)
        city_name.append(city_weather['name'])
        Lat.append(city_weather['coord']['lat'])
        Long.append(city_weather['coord']['lon'])
        MaxTemp.append(city_weather['main']['temp_max'])
        Humidity.append(city_weather['main']['humidity'])
        Cloundiness.append(city_weather['clouds']['all'])
        WindSpeed.append(city_weather['wind']['speed'])
      
    except: 
        print("City not found")
    
    time.sleep(1)
print(city_name) 
    

City :  upernavik
{'coord': {'lon': -56.15, 'lat': 72.79}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 269.75, 'feels_like': 265.3, 'temp_min': 269.75, 'temp_max': 269.75, 'pressure': 1009, 'humidity': 95, 'sea_level': 1009, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 2.77, 'deg': 128}, 'clouds': {'all': 88}, 'dt': 1603317927, 'sys': {'country': 'GL', 'sunrise': 1603280921, 'sunset': 1603307357}, 'timezone': -7200, 'id': 3418910, 'name': 'Upernavik', 'cod': 200}
City :  punta arenas
{'coord': {'lon': -70.92, 'lat': -53.15}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 282.09, 'feels_like': 277.95, 'temp_min': 282.04, 'temp_max': 282.15, 'pressure': 1007, 'humidity': 54}, 'visibility': 10000, 'wind': {'speed': 3.1, 'deg': 310}, 'clouds': {'all': 90}, 'dt': 1603317964, 'sys': {'type': 1, 'id': 8487, 'cou

City :  maceio
{'coord': {'lon': -35.74, 'lat': -9.67}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 299.15, 'feels_like': 301.26, 'temp_min': 299.15, 'temp_max': 299.15, 'pressure': 1015, 'humidity': 78}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 120}, 'clouds': {'all': 20}, 'dt': 1603318088, 'sys': {'type': 1, 'id': 8413, 'country': 'BR', 'sunrise': 1603266984, 'sunset': 1603311498}, 'timezone': -10800, 'id': 3395981, 'name': 'Maceió', 'cod': 200}
City :  new norfolk
{'coord': {'lon': 147.06, 'lat': -42.78}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 284.41, 'feels_like': 283.13, 'temp_min': 282.04, 'temp_max': 286.48, 'pressure': 1016, 'humidity': 76}, 'visibility': 10000, 'wind': {'speed': 0.89, 'deg': 310}, 'clouds': {'all': 97}, 'dt': 1603317860, 'sys': {'type': 3, 'id': 2031307, 'country': 'AU', 'sunrise':

City :  port alfred
{'coord': {'lon': 26.89, 'lat': -33.59}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 290.67, 'feels_like': 288.41, 'temp_min': 290.67, 'temp_max': 290.67, 'pressure': 1022, 'humidity': 85, 'sea_level': 1022, 'grnd_level': 1019}, 'visibility': 10000, 'wind': {'speed': 5.52, 'deg': 54}, 'clouds': {'all': 0}, 'dt': 1603317975, 'sys': {'country': 'ZA', 'sunrise': 1603336946, 'sunset': 1603384260}, 'timezone': 7200, 'id': 964432, 'name': 'Port Alfred', 'cod': 200}
City :  padang
{'coord': {'lon': 100.35, 'lat': -0.95}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'heavy intensity rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 297.22, 'feels_like': 300.05, 'temp_min': 297.22, 'temp_max': 297.22, 'pressure': 1010, 'humidity': 87, 'sea_level': 1010, 'grnd_level': 1010}, 'visibility': 2538, 'wind': {'speed': 2.49, 'deg': 21}, 'rain': {'1h': 7.488}, 'clouds': {'all': 100}, '

City :  kitimat
{'coord': {'lon': -128.65, 'lat': 54.05}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 278.71, 'feels_like': 276.46, 'temp_min': 278.71, 'temp_max': 278.71, 'pressure': 1025, 'humidity': 82}, 'visibility': 10000, 'wind': {'speed': 1, 'deg': 1}, 'clouds': {'all': 86}, 'dt': 1603318122, 'sys': {'type': 3, 'id': 2004800, 'country': 'CA', 'sunrise': 1603293305, 'sunset': 1603329770}, 'timezone': -25200, 'id': 5993072, 'name': 'Kitimat', 'cod': 200}
City :  charters towers
{'coord': {'lon': 146.27, 'lat': -20.1}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 296.77, 'feels_like': 295.83, 'temp_min': 296.77, 'temp_max': 296.77, 'pressure': 1013, 'humidity': 69, 'sea_level': 1013, 'grnd_level': 978}, 'visibility': 10000, 'wind': {'speed': 5.09, 'deg': 17}, 'clouds': {'all': 22}, 'dt': 1603318123, 'sys': {'country': 

City :  turukhansk
{'coord': {'lon': 87.98, 'lat': 65.82}, 'weather': [{'id': 600, 'main': 'Snow', 'description': 'light snow', 'icon': '13n'}], 'base': 'stations', 'main': {'temp': 267.92, 'feels_like': 263.62, 'temp_min': 267.92, 'temp_max': 267.92, 'pressure': 997, 'humidity': 98, 'sea_level': 997, 'grnd_level': 991}, 'visibility': 309, 'wind': {'speed': 2.34, 'deg': 227}, 'snow': {'1h': 0.17}, 'clouds': {'all': 97}, 'dt': 1603318142, 'sys': {'country': 'RU', 'sunrise': 1603330029, 'sunset': 1603361856}, 'timezone': 25200, 'id': 1488903, 'name': 'Turukhansk', 'cod': 200}
City :  bocaranga
{'cod': '404', 'message': 'city not found'}
City not found
City :  saint george
{'coord': {'lon': -113.58, 'lat': 37.1}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 300.58, 'feels_like': 296.73, 'temp_min': 299.15, 'temp_max': 301.48, 'pressure': 1012, 'humidity': 10}, 'visibility': 10000, 'wind': {'speed': 1.5, 'deg': 2

...

City :  rio grande
{'coord': {'lon': -52.1, 'lat': -32.03}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 289.26, 'feels_like': 289.49, 'temp_min': 289.26, 'temp_max': 289.26, 'pressure': 1023, 'humidity': 91}, 'visibility': 10000, 'wind': {'speed': 1.79, 'deg': 73, 'gust': 4.92}, 'clouds': {'all': 100}, 'dt': 1603318406, 'sys': {'type': 3, 'id': 2020865, 'country': 'BR', 'sunrise': 1603269664, 'sunset': 1603316670}, 'timezone': -10800, 'id': 3451138, 'name': 'Rio Grande', 'cod': 200}
City :  lilongwe
{'coord': {'lon': 33.79, 'lat': -13.97}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 296.54, 'feels_like': 294.67, 'temp_min': 296.54, 'temp_max': 296.54, 'pressure': 1014, 'humidity': 40, 'sea_level': 1014, 'grnd_level': 902}, 'visibility': 10000, 'wind': {'speed': 2.36, 'deg': 127}, 'clouds': {'all': 25}, 'dt': 1603318

City :  dudinka
{'coord': {'lon': 86.18, 'lat': 69.41}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 262.15, 'feels_like': 253.99, 'temp_min': 262.15, 'temp_max': 262.15, 'pressure': 995, 'humidity': 85}, 'visibility': 2500, 'wind': {'speed': 7, 'deg': 40}, 'clouds': {'all': 40}, 'dt': 1603318686, 'sys': {'type': 1, 'id': 8951, 'country': 'RU', 'sunrise': 1603331682, 'sunset': 1603361067}, 'timezone': 25200, 'id': 1507116, 'name': 'Dudinka', 'cod': 200}
City :  yunjinghong
{'cod': '404', 'message': 'city not found'}
City not found
City :  kirs
{'coord': {'lon': 52.24, 'lat': 59.34}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 270.85, 'feels_like': 265.94, 'temp_min': 270.85, 'temp_max': 270.85, 'pressure': 1009, 'humidity': 93, 'sea_level': 1009, 'grnd_level': 989}, 'visibility': 10000, 'wind': {'speed': 3.56, 'deg': 296}, 'clouds

City :  kuusamo
{'coord': {'lon': 29.18, 'lat': 65.97}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 270.15, 'feels_like': 263.97, 'temp_min': 270.15, 'temp_max': 270.15, 'pressure': 1010, 'humidity': 86}, 'visibility': 10000, 'wind': {'speed': 5.1, 'deg': 120}, 'clouds': {'all': 90}, 'dt': 1603318706, 'sys': {'type': 1, 'id': 1352, 'country': 'FI', 'sunrise': 1603344218, 'sunset': 1603375888}, 'timezone': 10800, 'id': 649924, 'name': 'Kuusamo', 'cod': 200}
City :  eyrarbakki
{'coord': {'lon': -21.15, 'lat': 63.86}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 276.3, 'feels_like': 269.28, 'temp_min': 275.37, 'temp_max': 277.15, 'pressure': 1002, 'humidity': 80}, 'visibility': 10000, 'wind': {'speed': 7.2, 'deg': 110}, 'clouds': {'all': 75}, 'dt': 1603318707, 'sys': {'type': 1, 'id': 91, 'country': 'IS', 'sunrise': 160326

City :  rapid valley
{'coord': {'lon': -103.15, 'lat': 44.06}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50d'}], 'base': 'stations', 'main': {'temp': 272.8, 'feels_like': 267.39, 'temp_min': 272.15, 'temp_max': 273.15, 'pressure': 1014, 'humidity': 92}, 'visibility': 4828, 'wind': {'speed': 4.6, 'deg': 60}, 'clouds': {'all': 90}, 'dt': 1603318725, 'sys': {'type': 1, 'id': 5623, 'country': 'US', 'sunrise': 1603286123, 'sunset': 1603324713}, 'timezone': -21600, 'id': 5768244, 'name': 'Rapid Valley', 'cod': 200}
City :  santa maria
{'coord': {'lon': -53.81, 'lat': -29.68}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 294.3, 'feels_like': 292.37, 'temp_min': 294.3, 'temp_max': 294.3, 'pressure': 1018, 'humidity': 76, 'sea_level': 1018, 'grnd_level': 1003}, 'visibility': 10000, 'wind': {'speed': 6.01, 'deg': 114}, 'clouds': {'all': 12}, 'dt': 1603318727, 'sys': {'country': 'BR', 'su

City :  denpasar
{'coord': {'lon': 115.22, 'lat': -8.65}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 298.15, 'feels_like': 304.22, 'temp_min': 298.15, 'temp_max': 298.15, 'pressure': 1009, 'humidity': 100}, 'visibility': 10000, 'wind': {'speed': 0.5, 'deg': 0}, 'clouds': {'all': 40}, 'dt': 1603318344, 'sys': {'type': 1, 'id': 9333, 'country': 'ID', 'sunrise': 1603317190, 'sunset': 1603361622}, 'timezone': 28800, 'id': 1645528, 'name': 'Denpasar', 'cod': 200}
City :  mtwara
{'coord': {'lon': 40.18, 'lat': -10.27}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 297.15, 'feels_like': 299.97, 'temp_min': 297.15, 'temp_max': 297.15, 'pressure': 1014, 'humidity': 88}, 'visibility': 10000, 'wind': {'speed': 2.6, 'deg': 180}, 'clouds': {'all': 20}, 'dt': 1603318743, 'sys': {'type': 1, 'id': 2617, 'country': 'TZ', 'sunrise': 160333

In [50]:
requests.get(query_url).json()['wind']

{'speed': 2.6, 'deg': 180}

In [51]:
# Get data from csv file
data = requests.get(query_url).json()
data




{'coord': {'lon': 40.18, 'lat': -10.27},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02n'}],
 'base': 'stations',
 'main': {'temp': 297.15,
  'feels_like': 299.97,
  'temp_min': 297.15,
  'temp_max': 297.15,
  'pressure': 1014,
  'humidity': 88},
 'visibility': 10000,
 'wind': {'speed': 2.6, 'deg': 180},
 'clouds': {'all': 20},
 'dt': 1603318743,
 'sys': {'type': 1,
  'id': 2617,
  'country': 'TZ',
  'sunrise': 1603335115,
  'sunset': 1603379712},
 'timezone': 10800,
 'id': 877747,
 'name': 'Mtwara',
 'cod': 200}

In [52]:
 Humidity

[95,
 54,
 83,
 94,
 75,
 84,
 46,
 48,
 70,
 99,
 77,
 60,
 72,
 81,
 51,
 78,
 76,
 93,
 58,
 80,
 94,
 92,
 73,
 54,
 79,
 64,
 80,
 35,
 80,
 51,
 85,
 87,
 84,
 90,
 63,
 60,
 85,
 90,
 68,
 73,
 60,
 96,
 71,
 74,
 69,
 82,
 69,
 60,
 70,
 78,
 69,
 98,
 70,
 75,
 88,
 88,
 69,
 58,
 15,
 67,
 98,
 10,
 87,
 47,
 97,
 74,
 81,
 70,
 69,
 77,
 60,
 54,
 88,
 79,
 85,
 82,
 96,
 96,
 97,
 86,
 93,
 76,
 88,
 100,
 72,
 96,
 73,
 74,
 70,
 83,
 81,
 100,
 94,
 52,
 81,
 63,
 60,
 58,
 58,
 95,
 60,
 67,
 94,
 63,
 91,
 100,
 34,
 79,
 45,
 68,
 98,
 91,
 93,
 91,
 84,
 73,
 89,
 59,
 76,
 13,
 80,
 94,
 93,
 83,
 74,
 63,
 84,
 95,
 76,
 65,
 15,
 88,
 97,
 88,
 78,
 60,
 58,
 55,
 88,
 94,
 97,
 93,
 77,
 68,
 80,
 89,
 75,
 70,
 89,
 66,
 72,
 96,
 100,
 78,
 87,
 74,
 58,
 97,
 58,
 73,
 63,
 66,
 72,
 64,
 66,
 93,
 84,
 58,
 97,
 86,
 12,
 82,
 72,
 74,
 62,
 67,
 91,
 92,
 73,
 73,
 29,
 43,
 84,
 94,
 74,
 87,
 86,
 77,
 20,
 72,
 87,
 96,
 72,
 97,
 79,
 85,
 99,
 53,
 77,
 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [79]:
#data.to_csv('.../WeatherPy')

#data = requests.get(query_url).json()



city_weather_df = pd.DataFrame({
    'city' : city_name,
    'Lat' : Lat,
    'Long' :Long,
    'MaxTemp' : MaxTemp,
    'Humidity' : Humidity,
   # 'Cloundiness' : Clound
   # 'Wind Speed' : WindSpeed
   # 'Date' : Date
})

#city_weather_df(data)
#city_weather_df = pd.DataFrame(data.json())
city_weather_df = city_weather_df.to_csv("cities.csv")
#city_weather_df.head()
city_weather_df
   


In [76]:
city_weather_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [77]:
#  Get the indices of cities that have humidity over 100%.
city_weather_df['humidity'] > 100

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
city_weather_df.drop['humidity'] > 100 = clean_city_data

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression